# ਨਮੂਨਾ 01: OpenAI SDK ਰਾਹੀਂ ਤੇਜ਼ ਗੱਲਬਾਤ

ਇਹ ਨੋਟਬੁੱਕ OpenAI SDK ਨੂੰ Microsoft Foundry Local ਨਾਲ ਤੇਜ਼ ਗੱਲਬਾਤ ਲਈ ਵਰਤਣ ਦਾ ਪ੍ਰਦਰਸ਼ਨ ਕਰਦੀ ਹੈ।

## ਝਲਕ

ਇਸ ਉਦਾਹਰਨ ਵਿੱਚ ਦਿਖਾਇਆ ਗਿਆ ਹੈ:
- Foundry Local ਨਾਲ OpenAI Python SDK ਦੀ ਵਰਤੋਂ
- Azure OpenAI ਅਤੇ ਸਥਾਨਕ Foundry ਸੰਰਚਨਾਵਾਂ ਨੂੰ ਸੰਭਾਲਣਾ
- ਸਹੀ ਤਰ੍ਹਾਂ ਗਲਤੀ ਸੰਭਾਲਣ ਅਤੇ ਫਾਲਬੈਕ ਰਣਨੀਤੀਆਂ ਨੂੰ ਲਾਗੂ ਕਰਨਾ
- ਸੇਵਾ ਪ੍ਰਬੰਧਨ ਲਈ FoundryLocalManager ਦੀ ਵਰਤੋਂ


## ਪੂਰਵ ਸ਼ਰਤਾਂ

ਯਕੀਨੀ ਬਣਾਓ ਕਿ ਤੁਹਾਡੇ ਕੋਲ:
1. Foundry Local ਇੰਸਟਾਲ ਅਤੇ ਚਲ ਰਹੀ ਹੈ
2. ਇੱਕ ਮਾਡਲ ਲੋਡ ਕੀਤਾ ਹੋਇਆ ਹੈ (ਜਿਵੇਂ, `foundry model run phi-4-mini`)
3. ਲੋੜੀਂਦੇ Python ਪੈਕੇਜ ਇੰਸਟਾਲ ਕੀਤੇ ਹੋਏ ਹਨ

### Dependencies ਇੰਸਟਾਲ ਕਰੋ


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## ਲੋੜੀਂਦੇ ਲਾਇਬ੍ਰੇਰੀਆਂ ਇਮਪੋਰਟ ਕਰੋ


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## ਕਨਫਿਗਰੇਸ਼ਨ

ਆਪਣੀ ਕਨਫਿਗਰੇਸ਼ਨ ਸੈਟ ਕਰੋ। ਤੁਸੀਂ ਹੇਠਾਂ ਦਿੱਤੇ ਵਿੱਚੋਂ ਕੋਈ ਇੱਕ ਚੋਣ ਕਰ ਸਕਦੇ ਹੋ:
1. **Azure OpenAI** (ਕਲਾਉਡ-ਅਧਾਰਿਤ)
2. **Foundry Local** (ਲੋਕਲ ਇਨਫਰੈਂਸ)

### ਵਿਕਲਪ 1: Azure OpenAI ਕਨਫਿਗਰੇਸ਼ਨ


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### ਵਿਕਲਪ 2: ਫਾਊਂਡਰੀ ਸਥਾਨਕ ਸੰਰਚਨਾ


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## ਕਲਾਇੰਟ ਸ਼ੁਰੂਆਤ

ਆਪਣੀ ਸੰਰਚਨਾ ਦੇ ਅਧਾਰ 'ਤੇ OpenAI ਕਲਾਇੰਟ ਬਣਾਓ:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## ਬੁਨਿਆਦੀ ਚੈਟ ਉਦਾਹਰਨ

ਆਓ ਇੱਕ ਸਧਾਰਨ ਚੈਟ ਇੰਟਰੈਕਸ਼ਨ ਦੀ ਜਾਂਚ ਕਰੀਏ:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## ਇੰਟਰਐਕਟਿਵ ਚੈਟ ਸੈਸ਼ਨ

ਮਾਡਲ ਦੇ ਜਵਾਬਾਂ ਦੇਖਣ ਲਈ ਵੱਖ-ਵੱਖ ਕਿਸਮ ਦੇ ਪ੍ਰੋੰਪਟ ਅਜ਼ਮਾਓ:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## ਉੱਚੇ ਵਰਤੋਂ: ਸਟ੍ਰੀਮਿੰਗ ਜਵਾਬ

ਲੰਬੇ ਜਵਾਬਾਂ ਲਈ, ਸਟ੍ਰੀਮਿੰਗ ਬਿਹਤਰ ਯੂਜ਼ਰ ਅਨੁਭਵ ਪ੍ਰਦਾਨ ਕਰ ਸਕਦੀ ਹੈ:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## ਗਲਤੀ ਸੰਭਾਲ ਅਤੇ ਨਿਦਾਨ

ਚਲੋ ਸੇਵਾ ਦੀ ਸਥਿਤੀ ਅਤੇ ਉਪਲਬਧ ਮਾਡਲਾਂ ਦੀ ਜਾਂਚ ਕਰੀਏ:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## ਕਸਟਮ ਪ੍ਰੌੰਪਟ ਟੈਸਟਿੰਗ

ਹੇਠਾਂ ਦਿੱਤੇ ਸੈੱਲ ਵਿੱਚ ਆਪਣੇ ਪ੍ਰੌੰਪਟ ਦੀ ਜਾਂਚ ਕਰੋ:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## ਸਾਰ

ਇਸ ਨੋਟਬੁੱਕ ਨੇ ਦਰਸਾਇਆ:

1. **✅ ਕਲਾਇੰਟ ਸੈਟਅਪ**: Foundry Local ਨਾਲ OpenAI SDK ਨੂੰ ਕਿਵੇਂ ਕਨਫਿਗਰ ਕਰਨਾ ਹੈ
2. **✅ ਬੇਸਿਕ ਚੈਟ**: ਸਧਾਰਨ ਰਿਕਵੈਸਟ-ਰਿਸਪਾਂਸ ਇੰਟਰੈਕਸ਼ਨ
3. **✅ ਸਟ੍ਰੀਮਿੰਗ**: ਰੀਅਲ-ਟਾਈਮ ਰਿਸਪਾਂਸ ਜਨਰੇਸ਼ਨ
4. **✅ ਐਰਰ ਹੈਂਡਲਿੰਗ**: ਮਜ਼ਬੂਤ ਐਰਰ ਹੈਂਡਲਿੰਗ ਅਤੇ ਡਾਇਗਨੋਸਟਿਕਸ
5. **✅ ਸਰਵਿਸ ਹੈਲਥ**: ਮਾਡਲ ਦੀ ਉਪਲਬਧਤਾ ਅਤੇ ਸਰਵਿਸ ਸਟੇਟਸ ਦੀ ਜਾਂਚ

### ਅਗਲੇ ਕਦਮ

- **ਸੈਂਪਲ 02**: Azure OpenAI ਸਪੋਰਟ ਨਾਲ ਉੱਨਤ SDK ਇੰਟੀਗ੍ਰੇਸ਼ਨ
- **ਸੈਂਪਲ 04**: ਇੱਕ Chainlit ਚੈਟ ਐਪਲੀਕੇਸ਼ਨ ਬਣਾਉਣਾ
- **ਸੈਂਪਲ 05**: ਮਲਟੀ-ਏਜੰਟ ਆਰਕੇਸਟ੍ਰੇਸ਼ਨ ਸਿਸਟਮ
- **ਸੈਂਪਲ 06**: ਸਮਰਥ ਮਾਡਲ ਰੂਟਿੰਗ

### Foundry Local ਦੇ ਮੁੱਖ ਫਾਇਦੇ

- 🔒 **ਪ੍ਰਾਈਵੇਸੀ**: ਡਾਟਾ ਕਦੇ ਵੀ ਤੁਹਾਡੇ ਡਿਵਾਈਸ ਤੋਂ ਬਾਹਰ ਨਹੀਂ ਜਾਂਦਾ
- ⚡ **ਸਪੀਡ**: ਘੱਟ ਲੈਟੈਂਸੀ ਵਾਲੀ ਲੋਕਲ ਇੰਫਰੈਂਸ
- 💰 **ਲਾਗਤ**: ਕੋਈ API ਵਰਤੋਂ ਦੀ ਲਾਗਤ ਨਹੀਂ
- 🔌 **ਆਫਲਾਈਨ**: ਇੰਟਰਨੈਟ ਕਨੈਕਸ਼ਨ ਤੋਂ ਬਿਨਾਂ ਕੰਮ ਕਰਦਾ ਹੈ
- 🛠️ **ਕੰਪੈਟਿਬਿਲਿਟੀ**: OpenAI-ਅਨੁਕੂਲ API
